In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
def gs_crit_full(df):
    overfit = np.abs(df.train_auc-df.val_auc) + np.abs(df.train_auc-df.test_auc) + \
              np.abs(df.train_auc-df.test_intra_auc)
    best_perf = 1-df.best_train_auc + 1-df.best_val_auc + 1-df.best_test_intra_auc + \
                1-df.best_test_auc
    
    return overfit + 2*best_perf

# S.or.

In [10]:
bdd_sor = pd.read_csv("/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/S.or./bdd_Sor_both.csv", index_col=0)

bdd_sor['crit'] = gs_crit_full(bdd_sor)
bdd_sor['min_best_perf'] = bdd_sor[['best_train_auc', 'best_val_auc', 'best_test_intra_auc', 'best_test_auc']].apply(min, axis=1)
bdd_sor['min_best_intra'] = bdd_sor[['best_train_auc', 'best_val_auc', 'best_test_intra_auc']].apply(min, axis=1)

# sort by criterion
bdd_sor.sort_values(by='crit', inplace=True)

bdd_sor

,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
36,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.579160,0.604990,0.446542,0.553264,0.358928,0.496460,3,0.008309,1440.647949,3.632089,0.446542,0.553264
42,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.559445,0.644491,0.470855,0.491379,0.220095,0.318655,4,0.117043,1624.265869,3.667661,0.470855,0.491379
30,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.451216,0.523493,0.518121,0.470443,0.539439,0.604158,0.560254,0.536022,0.035590,0.085306,1,0.036188,1.941209,3.678664,0.536022,0.536022
13,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.553424,0.569231,0.465871,0.523399,0.582868,0.611227,0.498037,0.529249,0.184135,0.079676,5,0.104995,1.337530,3.690624,0.498037,0.529249
59,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.577689,0.557173,0.484597,0.539717,0.573310,0.605405,0.505738,0.540948,0.047665,0.640966,3,0.061225,1.357648,3.700777,0.505738,0.540948
14,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.544938,0.552183,0.479236,0.494458,0.600027,0.601663,0.432045,0.577278,0.171240,0.044248,5,0.102017,1.357341,3.701402,0.432045,0.577278
8,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.428137,0.377547,0.557913,0.433190,0.592253,0.612058,0.455301,0.575123,0.187572,0.052213,1,0.258847,1.352152,3.715949,0.455301,0.575123
44,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.576009,0.550104,0.538810,0.543719,0.579782,0.602495,0.477952,0.525862,0.062806,0.234681,2,0.169498,1.341864,3.723213,0.477952,0.525862
47,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.527983,0.480665,0.484144,0.507543,0.578702,0.602079,0.437330,0.565271,0.395445,0.076689,3,0.187084,5.195068,3.744833,0.437330,0.565271
9,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.465016,0.437838,0.495772,0.521860,0.575460,0.613721,0.489731,0.501693,0.180442,0.087231,3,0.022244,1.342444,3.753565,0.489731,0.501693


In [11]:
print(bdd_sor[bdd_sor.min_best_perf >= 0.5].shape[0])
bdd_sor[bdd_sor.min_best_perf >= 0.5]

5


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
30,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.451216,0.523493,0.518121,0.470443,0.539439,0.604158,0.560254,0.536022,0.035590,0.085306,1,0.036188,1.941209,3.678664,0.536022,0.536022
59,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.577689,0.557173,0.484597,0.539717,0.573310,0.605405,0.505738,0.540948,0.047665,0.640966,3,0.061225,1.357648,3.700777,0.505738,0.540948
40,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.581048,0.518087,0.461492,0.502001,0.557051,0.556757,0.504228,0.616687,0.154609,0.001727,8,0.175016,1.334081,3.792117,0.504228,0.556757
28,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.637555,0.523909,0.461190,0.558190,0.605378,0.557588,0.500151,0.519397,0.218135,0.001597,5,0.151247,1.335749,4.004348,0.500151,0.519397
58,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.614237,0.488565,0.444428,0.454280,0.550616,0.538046,0.517970,0.511084,0.071475,0.004042,9,0.155039,1.328974,4.220007,0.511084,0.511084


In [13]:
print(bdd_sor[bdd_sor.min_best_intra >= 0.55].shape[0])
bdd_sor[bdd_sor.min_best_intra >= 0.55]

7


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
36,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.579160,0.604990,0.446542,0.553264,0.358928,0.496460,3,0.008309,1440.647949,3.632089,0.446542,0.553264
14,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.544938,0.552183,0.479236,0.494458,0.600027,0.601663,0.432045,0.577278,0.171240,0.044248,5,0.102017,1.357341,3.701402,0.432045,0.577278
8,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.428137,0.377547,0.557913,0.433190,0.592253,0.612058,0.455301,0.575123,0.187572,0.052213,1,0.258847,1.352152,3.715949,0.455301,0.575123
47,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.527983,0.480665,0.484144,0.507543,0.578702,0.602079,0.437330,0.565271,0.395445,0.076689,3,0.187084,5.195068,3.744833,0.437330,0.565271
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.556713,0.610603,0.461870,0.574046,0.567682,0.602079,0.467834,0.558805,0.198094,0.046071,5,0.079883,1.621132,3.773264,0.467834,0.558805
40,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.581048,0.518087,0.461492,0.502001,0.557051,0.556757,0.504228,0.616687,0.154609,0.001727,8,0.175016,1.334081,3.792117,0.504228,0.556757
35,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.904493,0.498545,0.506342,0.667796,0.857813,0.633680,0.480973,0.624384,0.010431,0.003729,3,0.007851,1.079954,3.847098,0.480973,0.624384


# S.T.s.

In [16]:
bdd_sts = pd.read_csv("/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/S.T.s./bdd_STs_both.csv", index_col=0)

bdd_sts['crit'] = gs_crit_full(bdd_sts)
bdd_sts['min_best_perf'] = bdd_sts[['best_train_auc', 'best_val_auc', 'best_test_intra_auc', 'best_test_auc']].apply(min, axis=1)
bdd_sts['min_best_intra'] = bdd_sts[['best_train_auc', 'best_val_auc', 'best_test_intra_auc']].apply(min, axis=1)

# sort by criterion
bdd_sts.sort_values(by='crit', inplace=True)

bdd_sts

,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
20,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.599778,0.610395,0.588946,0.562808,0.582046,0.620374,0.590607,0.542796,0.027290,0.008931,4,0.384765,1.359767e+00,3.386774,0.542796,0.542796
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.719751,0.515593,0.542585,0.744766,0.617458,0.599168,0.582301,0.668719,0.164393,0.000475,5,0.031012,1.258531e+00,3.471044,0.582301,0.599168
10,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.613891,0.546778,0.483540,0.642857,0.652167,0.594595,0.471761,0.658251,0.010350,0.002817,4,0.012746,1.344729e+00,3.472884,0.471761,0.594595
37,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.566634,0.511435,0.553760,0.588054,0.550280,0.611642,0.590003,0.555727,0.141279,0.027113,2,0.048906,1.347586e+00,3.474189,0.550280,0.550280
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.724171,0.521830,0.504077,0.728140,0.713138,0.528067,0.507098,0.713978,0.109223,0.000318,5,0.473691,1.278136e+00,3.501844,0.507098,0.528067
49,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.555038,0.582952,0.575053,0.523399,0.030362,0.152843,6,0.119353,1.043193e+04,3.527116,0.523399,0.523399
33,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500281,0.494387,0.495545,0.536330,0.522759,0.630353,0.574449,0.527709,0.154940,0.863166,2,0.311987,2.065082e+06,3.536138,0.522759,0.522759
41,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.447353,0.442827,0.443295,0.486145,0.536751,0.585863,0.588644,0.541564,0.124893,0.017749,9,0.350905,1.350551e+00,3.541732,0.536751,0.536751
39,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.610821,0.511850,0.582301,0.701663,0.554116,0.589189,0.586228,0.596983,0.192482,0.004856,0,0.029177,1.344788e+00,3.565301,0.554116,0.554116
54,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.514090,0.532225,0.567351,0.445197,0.522292,0.637422,0.557233,0.570505,0.171517,0.028050,1,0.369212,1.363224e+00,3.565383,0.522292,0.522292


In [17]:
print(bdd_sts[bdd_sts.min_best_perf >= 0.5].shape[0])
bdd_sts[bdd_sts.min_best_perf >= 0.5]

37


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
20,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.599778,0.610395,0.588946,0.562808,0.582046,0.620374,0.590607,0.542796,0.027290,0.008931,4,0.384765,1.359767e+00,3.386774,0.542796,0.542796
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.719751,0.515593,0.542585,0.744766,0.617458,0.599168,0.582301,0.668719,0.164393,0.000475,5,0.031012,1.258531e+00,3.471044,0.582301,0.599168
37,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.566634,0.511435,0.553760,0.588054,0.550280,0.611642,0.590003,0.555727,0.141279,0.027113,2,0.048906,1.347586e+00,3.474189,0.550280,0.550280
6,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.724171,0.521830,0.504077,0.728140,0.713138,0.528067,0.507098,0.713978,0.109223,0.000318,5,0.473691,1.278136e+00,3.501844,0.507098,0.528067
49,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.555038,0.582952,0.575053,0.523399,0.030362,0.152843,6,0.119353,1.043193e+04,3.527116,0.523399,0.523399
33,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500281,0.494387,0.495545,0.536330,0.522759,0.630353,0.574449,0.527709,0.154940,0.863166,2,0.311987,2.065082e+06,3.536138,0.522759,0.522759
41,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.447353,0.442827,0.443295,0.486145,0.536751,0.585863,0.588644,0.541564,0.124893,0.017749,9,0.350905,1.350551e+00,3.541732,0.536751,0.536751
39,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.610821,0.511850,0.582301,0.701663,0.554116,0.589189,0.586228,0.596983,0.192482,0.004856,0,0.029177,1.344788e+00,3.565301,0.554116,0.554116
54,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.514090,0.532225,0.567351,0.445197,0.522292,0.637422,0.557233,0.570505,0.171517,0.028050,1,0.369212,1.363224e+00,3.565383,0.522292,0.522292
45,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.500000,0.500000,0.500000,0.500000,0.517487,0.588358,0.589550,0.501539,0.076121,0.234943,2,0.347357,2.012086e+02,3.606132,0.501539,0.501539


In [18]:
print(bdd_sts[bdd_sts.min_best_intra >= 0.55].shape[0])
bdd_sts[bdd_sts.min_best_intra >= 0.55]

13


,model_path,train_auc,val_auc,test_auc,test_intra_auc,best_train_auc,best_val_auc,best_test_auc,best_test_intra_auc,drop_rate,lr,max_angle,sigma_noise,train_loss,crit,min_best_perf,min_best_intra
7,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.719751,0.515593,0.542585,0.744766,0.617458,0.599168,0.582301,0.668719,0.164393,0.000475,5,0.031012,1.258531,3.471044,0.582301,0.599168
10,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.613891,0.546778,0.483540,0.642857,0.652167,0.594595,0.471761,0.658251,0.010350,0.002817,4,0.012746,1.344729,3.472884,0.471761,0.594595
37,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.566634,0.511435,0.553760,0.588054,0.550280,0.611642,0.590003,0.555727,0.141279,0.027113,2,0.048906,1.347586,3.474189,0.550280,0.550280
39,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.610821,0.511850,0.582301,0.701663,0.554116,0.589189,0.586228,0.596983,0.192482,0.004856,0,0.029177,1.344788,3.565301,0.554116,0.554116
47,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.645805,0.518919,0.533525,0.762315,0.585041,0.587526,0.493506,0.701355,0.160540,0.007255,2,0.363501,1.296191,3.620820,0.493506,0.585041
40,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.486406,0.368399,0.424645,0.500000,0.558041,0.577963,0.581546,0.554187,0.088530,0.188025,0,0.225212,1.360515,3.649887,0.554187,0.554187
11,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.756400,0.537214,0.525974,0.718904,0.609911,0.598753,0.579432,0.582512,0.036740,0.000459,5,0.062965,1.231797,3.745892,0.579432,0.582512
22,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.849605,0.518503,0.531108,0.654557,0.641476,0.606237,0.542736,0.702278,0.005153,0.002315,4,0.023665,0.996867,3.859194,0.542736,0.606237
9,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.694704,0.491060,0.531410,0.756158,0.565667,0.590437,0.526125,0.596983,0.013489,0.006996,5,0.456686,1.287550,3.869969,0.526125,0.565667
2,/neurospin/dico/agaudin/Runs/09_new_repo/Outpu...,0.992891,0.472765,0.546814,0.696121,0.682077,0.576299,0.528994,0.766933,0.131382,0.000258,1,0.167687,0.885866,4.154364,0.528994,0.576299
